In [2]:
import tensorflow as tf
from tensorflow import keras

keras.__version__

'2.12.0'

In [21]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    model.compile(loss=keras.losses.MSE, optimizer=keras.optimizers.legacy.SGD(learning_rate=0.01))
    return model



In [22]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
keras_reg

<ipython-input-22-9dabefbb1669>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [23]:
reg = build_model()
reg

In [24]:
reg